In [1]:
from adversarial.defences.adv_classifier import *
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tqdm import tqdm

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
import numpy as np

In [4]:
# dataloader
from adversarial.utils import dataloader
from adversarial.attacks import random_attack

In [5]:
BATCH_SIZE = 16
EPSILONS = np.arange(0.01, 0.5, step=0.02)
ETA = 1e-5
EPOCHS = 6

In [6]:
train_data_loader = dataloader.DataLoader('./data', 16, training=True)
test_data_loader = dataloader.DataLoader('./data', 16, training= False)

In [7]:
from adversarial.defences import EnsembleModel
from adversarial.models import *

random_attack(앙상블 모델 넣을건데, x_image, adversarial_label, epsilon은 배열로)  
epsilon = np.arange(0.01,0.5,0.02)  
random_attack은 adversarial image를 리턴해준다. 

In [8]:
target_model = AdvClassifier()

In [9]:
# train

criterion = losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = ETA)

train_losses = tf.keras.metrics.CategoricalCrossentropy()
train_metrics = tf.keras.metrics.CategoricalAccuracy()

test_losses = tf.keras.metrics.CategoricalCrossentropy()
test_metrics = tf.keras.metrics.CategoricalAccuracy()

In [10]:
ensemble_model = EnsembleModel([
    VGG19, VGG16, MobileNet, MobileNetV2, ResNet50, ResNet50V2
], devices = [0])

In [11]:
labels = [[0,1] ,[1,0]]
    
y_batch = []
    
    
for label in labels : 
    for time in range(16) :
        y_batch.append(label)

y_tensor = tf.convert_to_tensor(y_batch)

In [12]:
def train_step(images, adv_labels):

    adv, _, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    with tf.GradientTape() as tape:
        outputs = target_model(x_batch, training = True)
      #  print('y_batch : ' ,y_batch)
       # print('outputs : ' ,outputs)
        loss = criterion(y_batch, outputs)
    #print(loss.numpy())
    train_losses.update_state(y_batch, outputs)
    train_metrics.update_state(y_batch, outputs)
        
    grads = tape.gradient(loss, target_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, target_model.trainable_variables))

In [ ]:
target_model.trainable_variables

In [13]:
def test_step(images, labels, adv_labels) :
    adv,_, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    #y_batch = tf.concat([tf.zeros(16, dtype = tf.int32),
     #                    tf.ones(16, dtype = tf.int32)], 
      #                   axis = 0)
    
    predictions = target_model(x_batch, training= False)
    loss = tf.keras.losses.binary_crossentropy(y_batch, predictions)
    test_losses.update_state(y_batch, predictions)
    test_metrics.update_state(y_batch, predictions)

In [14]:
def train():

    train_loss_list = []
    test_loss_list = []

    for e in range(EPOCHS):
        for x_batch, y_batch, adv_batch in tqdm(iter(train_data_loader)):
            train_step(x_batch, adv_batch)

        for x_batch, y_batch, adv_batch in tqdm(iter(test_data_loader)):
            test_step(x_batch, y_batch, adv_batch)


        train_loss = train_losses.result()
        train_acc = train_metrics.result()

        test_loss = test_losses.result()
        test_acc = test_metrics.result()

        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)

        train_losses.reset_states()
        train_metrics.reset_states()
        test_losses.reset_states()
        test_metrics.reset_states()
        
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [15]:
def load_custom_weights(target, path):
    with open(path, "rb") as f:
        data = pickle.load(f)
        target.load_weights(data)

def save_custom_weights(target, path):
    with open(path, "wb") as f:
        pickle.dump(target.get_weights(), f)

In [ ]:
target_model.summary()

In [16]:
train()

0it [00:00, ?it/s]

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


806it [31:25,  2.34s/it]
32it [00:48,  1.52s/it]
0it [00:00, ?it/s]

Epochs 1/6, train loss: 0.62572742, train acc: 0.6074, test loss: 0.58062869, test acc: 0.6621


806it [33:13,  2.47s/it]
32it [02:28,  4.63s/it]
1it [00:00,  8.42it/s]

...

Epochs 4/6, train loss: 0.56420678, train acc: 0.6711, test loss: 0.56936163, test acc: 0.6621


806it [30:09,  2.24s/it]
32it [01:23,  2.59s/it]
0it [00:00, ?it/s]

Epochs 5/6, train loss: 0.57220560, train acc: 0.6616, test loss: 0.52034312, test acc: 0.7207


806it [36:15,  2.70s/it]
32it [01:54,  3.56s/it]

Epochs 6/6, train loss: 0.56949013, train acc: 0.6617, test loss: 0.57741475, test acc: 0.6572


In [17]:
save_custom_weights(target_model, './adv_classifier_BN_6.4.pkl')

In [ ]:
train()

806it [32:47,  2.44s/it]
32it [01:22,  2.58s/it]
1it [00:00,  6.30it/s]

Epochs 1/6, train loss: 0.56903332, train acc: 0.6626, test loss: 0.54330730, test acc: 0.6982


806it [32:23,  2.41s/it]
32it [00:55,  1.73s/it]
1it [00:00,  5.08it/s]

Epochs 2/6, train loss: 0.55978245, train acc: 0.6762, test loss: 0.59399581, test acc: 0.6436


266it [12:00,  2.42s/it]